In [21]:
import os
from datetime import datetime
import smtplib

import email.utils
import email.header

from email.message import EmailMessage

Тут я просто объявляю логин и пароль, но на самом деле так делать не очень хорошо и нужно писать в какой-нибудь скрытый файл credentials.json как с Google API. 

Просто пароль от почты не подойдет, MAIL забанит такую попытку как попытку взлома. 

Нужна генерация временного пароля! 

Для этого [следуем гайду](https://help.mail.ru/mail/security/protection/external). Да, придется привязать номер телефона, но иначе никак. Если будут сложности, зовите, помогу разобраться. 

Здесь сразу виден наш временный пароль от Сэнгера, просьба его не использовать :D

In [33]:
EMAIL_ADDRESS = 'oligosynthesis@sysbiomed.ru'
EMAIL_PASSWORD = 'Lv5wAEiVL1ztsCgUuZN4'

Иногда почты записываются в виде 

`Ivan Ivanov <ivan@ivanov.com>`

А иногда записываются в виде 

`ivan@ivanov.com`

А ещё иногда ещё хуже 

`Иван Ивано <ivan@ivanov.com>`

Чтобы в любом таком случае письмо отправилось, адрес нужно перезаписать в правильной кодировке. К счастью, это уже давно решенный вопрос.  

In [23]:
def reformat_address(s: str) -> str:
    """
    Функция для форматирования имен адресатов и их почтовых адресов, честно
    позаимствована со Stackoverflow. \n \n
    :param s: строка для форматирования
    :return: отформатированная подходящим образом строка
    """
    # parse email to get user real name and email address
    name, address = email.utils.parseaddr(s)
    # encode the user name use utf-8 to avoid encoding error
    name_encoded = (email.header.Header(name, 'utf-8')).encode()
    # construct the email address again
    return email.utils.formataddr((name_encoded, address))

Прочитаем файл с адресатами

In [24]:
# формально вы можете передавать его как вам удобно 

with open("unique_emails.txt", "r") as fr:
    recipients_list = fr.read().splitlines()

Создадим какой-то список приложений к письму. 

Важно помнить, что у каждого почтового агента свои ограничения по размерам, но несколько мегабайт точно отправятся.

In [25]:
# вот тут тоже, можете каждому письму что-то свое придумать

attachments_list = [
    "attachments/some_tables.tsv",
    "attachments/some_logs.txt",
    "attachments/some_reports.pdf"
]

Просто отправлять письмо скучно, сделаем какой-то динамичный шаблон для него. 

In [26]:
# прочитаем наш шаблон письма
# тут очень важно заранее соотнести все переменные,
# чтобы потом не получить нелепых ошибок имен

with open("mail.template", "r", encoding="utf-8") as fr:
    template = fr.read()

In [27]:
print(template)

f"""Привет! Это тестовое письмо, не пугайтесь. 

Это отчет команды автоматического синтеза!

Отчет сформимрован {current_timestamp}, всего было синтезировано {oligos_val} олигов.

Можно забрать в плашке с именем {plate_name}, лунки смотри в приложении в файле {plate_file}.
"""



Как мы видим, в файле все записано как f-строка, то есть в фигурных скобках указаны имена переменных текущего исполняемого кода. 

Важно следить, чтобы они все существовали и были строками. 

Объявим их!

In [28]:
current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M')
oligos_val = "455"
plate_name = "OFVV322"
plate_file = "some_tables.tsv"

А теперь мы можем исполнить эту строку, как будто бы она строка кода

In [29]:
type(eval(template))

str

In [30]:
print(eval(template))

Привет! Это тестовое письмо, не пугайтесь. 

Это отчет команды автоматического синтеза!

Отчет сформимрован 2023-10-10 15:37, всего было синтезировано 455 олигов.

Можно забрать в плашке с именем OFVV322, лунки смотри в приложении в файле some_tables.tsv.



Вот такое уже можно передавать в объект письма. 

Создаем тело письма

In [34]:
msg = EmailMessage()

msg['Subject'] = 'This is an example mail'  # это тема письма, тоже можно подставить какое-то имя синтеза
msg['From'] = EMAIL_ADDRESS
# используем функцию и превращаем все в список адресатов 
# с корректными и точно кодируемыми адресамми
# это нужно на случай идиотских имен и форматов 
msg['To'] = ', '.join(list(map(reformat_address, recipients_list)))

# тут текст письма, который скомпилировали ранее
msg.set_content(eval(template))

# прикрепляем файлы
for elem in attachments_list:
    with open(elem, 'rb') as frb:  # главное открывать бинарно файлы, иначе не отправятся
        msg.add_attachment(
            frb.read(),
            maintype='application',
            subtype='octet-stream',
            filename=os.path.split(elem)[1]  # вот тут извлекаем только имя файла, чтобы пути остались у нас
        )

И отправляем сразу всем получателям из файла. 

Тут сразу указаны нужные сервер и порт для нашей рабочей почты.

In [35]:
with smtplib.SMTP_SSL('smtp.mail.ru', 465) as smtp:
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD) 
    smtp.send_message(msg)

In [36]:
import smtplib

EMAIL_ADDRESS = "your_email@mail.ru"
EMAIL_PASSWORD = "your_application_password"

with smtplib.SMTP_SSL('smtp.mail.ru', 465) as smtp:
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    # Остальная часть кода для отправки сообщения


SMTPAuthenticationError: (535, b'5.7.0 NEOBHODIM parol prilozheniya https://help.mail.ru/mail/security/protection/external / Application password is REQUIRED')